# TrAdaBoost Transfer Learning ULSA -> UENG
# Model 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from scipy.stats import kstest
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

In [2]:
from transformers.features_missing_vals import imputation_pipeline
from transformers.ethnic_groups import CombineEthnicGroups

In [3]:
from transfer_learning.TrAdaBoost import TrAdaBoost

In [65]:
from model_evaluation.cross_val import direct_transfer_cv, TrAdaBoost_transfer_cv

# Params

In [5]:
source_school = 'ULSA'
target_school = 'UENG'

In [6]:
# Model features
cols = ['STDNT_FEMALE',
        'STDNT_AGE_BIN',
        'HS_GPA_Imp_Interp',
        'CURR_GPA',
        'HS_CALC_IND',
        'GROSS_FAM_INC',
        'SNGL_PRNT_IND',
        'STDNT_ASIAN_IND',
        'STDNT_BLACK_IND',
        'STDNT_HSPNC_IND',
        'STDNT_NTV_AMRCN_HWIAN_IND',
        'STDNT_MULTI_ETHNC_IND',
        'STDNT_ETHNC_NAN',
        'SAT_ACT_TOTAL_BIN',
        'PRNT_ED_LVL',
        'No_grades_at_all',
        'Grade_Overall_I_for_1_and_more_courses',
        'Grade_W_for_1_course',
        'Grade_W_for_2_courses',
        'Grade_W_for_3_and_more_courses',
        'Grade_NR_for_1_and_more_courses']

In [7]:
random_state = 1

In [8]:
clf = LogisticRegression(penalty=None, random_state=random_state, max_iter=10000)

In [9]:
# Parameters for fairness metrics

# Priviledged values of attributes
attribites_dic = {
    'STDNT_ETHNC_grouped': 0, # white
    'STDNT_FEMALE': 0 # male
    }

# The threshold for intervention is based on the percentile of predicted dropout probabilities, 
# specifically targeting the top 10% of students with the highest dropout probabilities
treshold=[10]

# Percentile values of predicted dropout probabilities that determine the risk ranking of a student
rank_thresholds = [90, 95, 97, 99, 99.5]

# Data

In [10]:
features_df = pd.read_csv('features_df.csv', low_memory=False)
source_df = features_df[features_df['PRMRY_CRER_CD']==source_school].copy()
target_df = features_df[features_df['PRMRY_CRER_CD']==target_school].copy()

# Reset target index to ensure that cross-validation indices will be accurately applied
target_df.reset_index(drop=True, inplace=True)

In [ ]:
# Group together all ethnic groups, excluding white and asian populations, as they lack a sufficient 
# number of dropouts for a meaningful fairness analysis

# Add new column 'STDNT_ETHNC_grouped' with the following values: 0 for white, 1 for asian, and 2 for other
combine_transformer = CombineEthnicGroups()
target_df = combine_transformer.fit_transform(target_df)

target_df.groupby('STDNT_ETHNC_grouped')[['y_ENROLLED_1_YEAR_LATER']].agg(['count', 'sum'])

In [12]:
# Impute missing values and apply WoE transformation for the direct transfer
source_df = combine_transformer.fit_transform(source_df)
source_imp = imputation_pipeline.fit_transform(source_df, source_df['y_ENROLLED_1_YEAR_LATER'])
target_imp = imputation_pipeline.transform(target_df)

# Train and test data sets in cross-validation
The target dataset will be partitioned into a training set comprising 33.3% of the data and a testing set comprising 66.6%.
For TrAdaBoost, the training set will encompass the source dataset and 33.3% of the target dataset, while the testing set for TrAdaBoost will consist of the remaining 66.6% from the target dataset. This specific partition is chosen because the testing set needs to contain a sufficient number of dropouts for the evaluation of AUC and other metrics. In cases with a low dropout count, the confidence interval of AUC tends to be broad, making the AUC estimate less reliable. Specifically, 66.6% of the target sample includes 84-85 instances of student dropouts.

In [13]:
stratified_split = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)
splits = stratified_split.split(target_df, target_df['y_ENROLLED_1_YEAR_LATER'])

In [14]:
# Generate indices for the training and testing datasets for every split in the cross-validation
train_target_indx = []
test_target_indx = []

for i, (test_index, train_index) in enumerate(splits):
    train_target_indx.append(train_index)
    test_target_indx.append(test_index)

In [ ]:
# Check the cross-validation splits
for i in range(stratified_split.get_n_splits()):
    train_size = len(train_target_indx[i])
    test_size = len(test_target_indx[i])
    train_dropout = target_df.loc[train_target_indx[i], 'y_ENROLLED_1_YEAR_LATER'].mean()
    test_dropout = target_df.loc[test_target_indx[i], 'y_ENROLLED_1_YEAR_LATER'].mean()
    test_dropout_num = target_df.loc[test_target_indx[i], 'y_ENROLLED_1_YEAR_LATER'].sum()
    print(f"Split {i}: Training set size={train_size}, testing set size={test_size}")
    print(f"Split {i}: Training dropout rate={train_dropout:.2%}, testing dropout rate={test_dropout:.2%},", 
          f"testing number of dropouts={test_dropout_num:.0f}")

# Direct transfer
To ensure comparability between direct transfer and TrAdaBoost, the evaluation of direct transfer will be conducted using identical cross-validation test sets as for TrAdaBoost. Using different samples to compare these methods may result in variations in model evaluation metrics, which could be attributed to sample differences rather than indicating the superiority of one model over the other.

## Source model

In [16]:
# Fit the source model
clf_source = clone(clf)
clf_source.fit(source_imp[cols], source_imp['y_ENROLLED_1_YEAR_LATER'])

LogisticRegression(max_iter=10000, penalty=None, random_state=1)

## Cross-validating the direct transfer of the source model
The performance of the source model 'clf_source' trained on the source dataset (refer to the cell above), is evaluated on three  target test sets, specified by the indices 'test_target_indx'. The evaluation includes classification accuracy and fairness metrics.

In [17]:
cv_res_direct = direct_transfer_cv(
    target_imp, # Target data set with imputed missing values
    test_target_indx, # Indices of the training sets for each cross-validation splits
    clf_source, cols, # Fitted source model and its features
    'Direct transfer', 
    attribites_dic, treshold, rank_thresholds) # Fairness metrics' parameters 

## Classification accuracy

### AUC

In [18]:
cv_res_direct['AUC']

,split_0,split_1,split_2,mean,se
Direct transfer,0.789893,0.807306,0.772405,0.789868,0.010075


### Pietra Index and Kolmogorov-Smirnov test

In [19]:
# Pietra index
cv_res_direct['pietra_index']

,split_0,split_1,split_2,mean,se
Direct transfer,0.481103,0.541203,0.443023,0.488443,0.028579


In [20]:
# KS p-value
cv_res_direct['KS_pvalue'].iloc[:, :3]

,split_0,split_1,split_2
Direct transfer,2.245806e-18,2.995791e-23,1.630568e-15


## Fairness metrics

### Sliced AUCs

In [21]:
cv_res_direct['sliced_AUCs']

,Attribute,Value,split_0,split_1,split_2,mean,se
0,STDNT_ETHNC_grouped,1,0.733733,0.746427,0.743339,0.741166,0.003822
1,STDNT_ETHNC_grouped,0,0.874387,0.852876,0.832646,0.853303,0.012051
2,STDNT_ETHNC_grouped,2,0.771485,0.830794,0.764897,0.789058,0.020954
3,STDNT_FEMALE,0,0.788091,0.785025,0.776988,0.783368,0.003311
4,STDNT_FEMALE,1,0.789758,0.88612,0.757039,0.810972,0.038743


### Equal opportunity difference

In [22]:
cv_res_direct['equal_opportunity_diff']

,Attribute,Value,split_0,split_1,split_2,mean,se
0,STDNT_ETHNC_grouped,1,-0.439394,-0.357317,-0.418182,-0.404964,0.024598
1,STDNT_ETHNC_grouped,0,0,0,0,0.000000,0.000000
2,STDNT_ETHNC_grouped,2,0.0,0.031813,0.078571,0.036795,0.022818
3,STDNT_FEMALE,0,0,0,0,0.000000,0.000000
4,STDNT_FEMALE,1,0.126923,0.093942,0.008185,0.076350,0.035387


### Generalized entropy index

In [23]:
cv_res_direct['gen_entropy']

,split_0,split_1,split_2,mean,se
generalized_entropy,0.404255,0.292453,0.465909,0.387539,0.050765
between_groups,0.060568,0.031204,0.075597,0.055790,0.013036


### Generalized entropy index that takes into account the risk ranks of students

In [24]:
cv_res_direct['gen_entropy_ranks']

,split_0,split_1,split_2,mean,se
generalized_entropy_ranks,0.565577,0.437151,0.630385,0.544371,0.056781
between_groups_ranks,0.040728,0.026243,0.073669,0.046880,0.014032


# TrAdaBoost

In [25]:
TrAdaBoost_model = TrAdaBoost(estimator=clone(clf), max_num_iterations=10)

## Cross-validating the TrAdaBoost
In every cross-validation split, the TrAdaBoost model is trained using the source dataset combined with 33.3% of the target dataset. Subsequently, the model is evaluated on the remaining 66.6% of the target dataset. The evaluation includes classification accuracy and fairness metrics.

In [26]:
cv_res_TrAdaBoost = TrAdaBoost_transfer_cv(
    source_df, target_df, # Source and target data sets
    train_target_indx, # Indices of the training sets for cross-validation splits
    test_target_indx, # Indices of the testing sets for cross-validation splits
    imputation_pipeline, # Pipeline to impute missing values and apply WoE transformation
    TrAdaBoost_model, 
    cols, # Model features
    'TrAdaBoost', 
    attribites_dic, treshold, rank_thresholds) # Fairness metrics' parameters 

## Classification accuracy

### AUC

In [27]:
pd.concat([cv_res_TrAdaBoost['AUC'], cv_res_direct['AUC']])

,split_0,split_1,split_2,mean,se
TrAdaBoost,0.800267,0.826088,0.768393,0.798249,0.016686
Direct transfer,0.789893,0.807306,0.772405,0.789868,0.010075


### Pietra index and Kolmogorov-Smirnov test

In [28]:
# Pietra index
pd.concat([cv_res_TrAdaBoost['pietra_index'], cv_res_direct['pietra_index']])

,split_0,split_1,split_2,mean,se
TrAdaBoost,0.511114,0.562943,0.482434,0.518830,0.023559
Direct transfer,0.481103,0.541203,0.443023,0.488443,0.028579


In [29]:
# KS p-value
pd.concat([cv_res_TrAdaBoost['KS_pvalue'].iloc[:,:3], 
           cv_res_direct['KS_pvalue'].iloc[:,:3]])

,split_0,split_1,split_2
TrAdaBoost,7.670561e-21,2.729856e-25,1.762516e-18
Direct transfer,2.245806e-18,2.995791e-23,1.630568e-15


## Fairness metrics

### Sliced AUCs

In [30]:
cv_res_TrAdaBoost['sliced_AUCs']

,Attribute,Value,split_0,split_1,split_2,mean,se
0,STDNT_ETHNC_grouped,1,0.716388,0.796827,0.721029,0.744748,0.026074
1,STDNT_ETHNC_grouped,0,0.879481,0.853756,0.830113,0.854450,0.014255
2,STDNT_ETHNC_grouped,2,0.777966,0.812742,0.764587,0.785098,0.014351
3,STDNT_FEMALE,0,0.793807,0.80356,0.770961,0.789443,0.009660
4,STDNT_FEMALE,1,0.801314,0.878339,0.75306,0.810904,0.036481


In [31]:
cv_res_TrAdaBoost['sliced_AUCs'][['Attribute', 'Value', 'mean', 'se']].merge(
    cv_res_direct['sliced_AUCs'][['Attribute', 'Value', 'mean', 'se']], 
    on=['Attribute', 'Value'],
    suffixes=('_TrAdaBoost', '_direct')
    )[['Attribute', 'Value', 'mean_TrAdaBoost', 'mean_direct', 'se_TrAdaBoost', 'se_direct']]

,Attribute,Value,mean_TrAdaBoost,mean_direct,se_TrAdaBoost,se_direct
0,STDNT_ETHNC_grouped,1,0.744748,0.741166,0.026074,0.003822
1,STDNT_ETHNC_grouped,0,0.854450,0.853303,0.014255,0.012051
2,STDNT_ETHNC_grouped,2,0.785098,0.789058,0.014351,0.020954
3,STDNT_FEMALE,0,0.789443,0.783368,0.009660,0.003311
4,STDNT_FEMALE,1,0.810904,0.810972,0.036481,0.038743


### Equal opportunity difference

In [32]:
cv_res_TrAdaBoost['equal_opportunity_diff']

,Attribute,Value,split_0,split_1,split_2,mean,se
0,STDNT_ETHNC_grouped,1,-0.231602,-0.108537,-0.401299,-0.247146,0.084870
1,STDNT_ETHNC_grouped,0,0,0,0,0.000000,0.000000
2,STDNT_ETHNC_grouped,2,0.071429,0.037116,-0.021429,0.029039,0.027108
3,STDNT_FEMALE,0,0,0,0,0.000000,0.000000
4,STDNT_FEMALE,1,-0.003846,0.005268,-0.05506,-0.017879,0.018775


In [33]:
cv_res_TrAdaBoost['equal_opportunity_diff'][['Attribute', 'Value', 'mean', 'se']].merge(
    cv_res_direct['equal_opportunity_diff'][['Attribute', 'Value', 'mean', 'se']], 
    on=['Attribute', 'Value'],
    suffixes=('_TrAdaBoost', '_direct')
    )[['Attribute', 'Value', 'mean_TrAdaBoost', 'mean_direct', 'se_TrAdaBoost', 'se_direct']]

,Attribute,Value,mean_TrAdaBoost,mean_direct,se_TrAdaBoost,se_direct
0,STDNT_ETHNC_grouped,1,-0.247146,-0.404964,0.084870,0.024598
1,STDNT_ETHNC_grouped,0,0.000000,0.000000,0.000000,0.000000
2,STDNT_ETHNC_grouped,2,0.029039,0.036795,0.027108,0.022818
3,STDNT_FEMALE,0,0.000000,0.000000,0.000000,0.000000
4,STDNT_FEMALE,1,-0.017879,0.076350,0.018775,0.035387


### Generalized entropy index

In [34]:
cv_res_TrAdaBoost['gen_entropy']

,split_0,split_1,split_2,mean,se
generalized_entropy,0.404255,0.277778,0.465909,0.382647,0.055373
between_groups,0.02184,0.003552,0.055098,0.026830,0.015088


In [35]:
cv_res_TrAdaBoost['gen_entropy'][['mean', 'se']].merge(
    cv_res_direct['gen_entropy'][['mean', 'se']], 
    left_index=True, right_index=True,
    suffixes=('_TrAdaBoost', '_direct')
    )[['mean_TrAdaBoost', 'mean_direct', 'se_TrAdaBoost', 'se_direct']]

,mean_TrAdaBoost,mean_direct,se_TrAdaBoost,se_direct
generalized_entropy,0.382647,0.387539,0.055373,0.050765
between_groups,0.026830,0.055790,0.015088,0.013036


### Generalized entropy index that takes into account the risk ranks of students

In [36]:
cv_res_TrAdaBoost['gen_entropy_ranks']

,split_0,split_1,split_2,mean,se
generalized_entropy_ranks,0.587053,0.415469,0.645606,0.549376,0.069054
between_groups_ranks,0.02794,0.010205,0.052998,0.030381,0.012414


In [37]:
cv_res_TrAdaBoost['gen_entropy_ranks'][['mean', 'se']].merge(
    cv_res_direct['gen_entropy_ranks'][['mean', 'se']], 
    left_index=True, right_index=True,
    suffixes=('_TrAdaBoost', '_direct')
    )[['mean_TrAdaBoost', 'mean_direct', 'se_TrAdaBoost', 'se_direct']]

,mean_TrAdaBoost,mean_direct,se_TrAdaBoost,se_direct
generalized_entropy_ranks,0.549376,0.544371,0.069054,0.056781
between_groups_ranks,0.030381,0.046880,0.012414,0.014032
